In [1]:
import pandas as pd

Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 

Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [5]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [60]:
grouped_users = ratings.groupby('userId').count().reset_index()
filtered_users = grouped_users[ (grouped_users['movieId'] > 99) ]

users_list = filtered_users['userId'].tolist()

filtered_ratings = ratings [ ratings['userId'].isin(users_list) ]
first_date = filtered_ratings[['userId', 'timestamp']].groupby('userId').min().reset_index()
last_date = filtered_ratings[['userId', 'timestamp']].groupby('userId').max().reset_index()

first_date.rename(columns={'timestamp': 'timestamp_first'}, inplace=True)
last_date.rename(columns={'timestamp': 'timestamp_last'}, inplace=True)

joined_table = first_date.merge(last_date)
joined_table['lifetime, days'] = round ((joined_table['timestamp_last'] - joined_table['timestamp_first']) /60 /60 /24 , 1)
joined_table.sort_values('lifetime, days', ascending = False).head()

,userId,timestamp_first,timestamp_last,"lifetime, days"
140,380,949367125,1465156786,5969.8
212,547,974777109,1476587644,5808.0
161,427,939332587,1415901843,5515.8
3,15,997937239,1469330735,5455.9
248,624,1019123866,1476616373,5295.1


Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 
Необходимо сформировать две таблицы:
+ таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
+ аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [65]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd.head(1)

,client_id,rzd_revenue
0,111,1093


In [66]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto.head(1)

,client_id,auto_revenue
0,113,57483


In [67]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air.head(1)

,client_id,air_revenue
0,115,81


In [68]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base.head(1)

,client_id,address
0,111,Комсомольская 4


In [77]:
print('---решение ниже---')

---решение ниже---


In [69]:
revenue_by_clients = rzd.merge(auto, how = 'outer').merge(air, how = 'outer')
revenue_by_clients

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [76]:
revenue_by_clients_addresses = client_base.merge(revenue_by_clients, how = 'outer')
revenue_by_clients_addresses

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


## Задача

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

***
### Решение
+ данные хранить в отдельном датасэте

+ Если для каждого посещения известна точная дата (timestamp) посещения, то добавить (заджойнить) информацию о координатах в таблицу визитов и покупок можно было бы по паре userId и timestamp

+ По timestamp можно предположить место работы/учебы и местоположение дома клиента

+ Можно посчитать частоту покупок по определенным категориям товаров из определенных локаций для каждого клиента, а также сегментировать все локации для всех клиентов и посчитать общую статистику для всех пар "локация" - "категория"

+ Если timestamp неизвестен, то можно сегментировать город, добавить в исходный датасэт сегменты и сделать свод в объединенном датасэте - в строках сегменты города, пользователи - в столбцах отдельно число визитов и число покапок по категориям

+ (дополнительно для городской компании с наружной рекламой) Можно сегментировать город и посчитать частоту визитов через определеные каналы в определенных районах, чтобы предпооложить качество охвата локальной рекламы
